In [0]:
tiers = ["bronze", "silver", "gold"]
s3_path = {tier: f"s3://{tier}bucketkrsna/" for tier in tiers}

bronze_s3 = s3_path["bronze"]
silver_s3 = s3_path["silver"]
gold_s3 = s3_path["gold"]

dbutils.fs.ls(bronze_s3)
dbutils.fs.ls(silver_s3)
dbutils.fs.ls(gold_s3)

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"


try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json().get('features', [])

    if not data:
        print("No Data returned for the specified date range")
    else:
        file_path = f"{bronze_s3}/{start_date}_earthquake.json"

        json_data = json.dumps(data, indent = 4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data Successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error Fetching data from api : {e}")

In [0]:
data[0]

In [0]:
output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "bronze_s3": bronze_s3,
    "silver_s3": silver_s3,
    "gold_s3": gold_s3
}

dbutils.jobs.taskValues.set(key="bronze_output", value=output_data)